In [ ]:
# Load image and segmentation mask
# Preprocess image (resize, normalize)

# Define U-Net model:
    # Encoder path (downsampling):
        # conv → relu → conv → relu → maxpool
        # repeat, doubling channels

    # Decoder path (upsampling):
        # upsample → concat with encoder feature map → convs
        # repeat, halving channels

    # Final 1x1 conv → class scores for each pixel

# Forward pass → compute loss → backprop → update

# Use trained model to predict segmentation masks
